# Extract frames from videos

At multiple points in the method, we rely on having access to each individual frame in the observation video (during object detection, drone movement calculations, and 3D landscape reconstruction). In this notebook we extract and save all frames from an observation and save them as individual image files

In [1]:
import os
import glob
import time

import cv2
from imutils.video import FileVideoStream
import json
import numpy as np

geladas_root = os.path.dirname(os.path.dirname(os.getcwd()))
local_paths_file = os.path.join(geladas_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

In [ ]:
# Where the videos are stored locally
# Extracted frames will also be stored in a subfolder called "frames/[video name]" 
base_folder = local_paths["videos_folder"]

# The name of the video we want to extract frames from 
# (can use glob patterns to specify multiple videos)
video_name = "DJI_0205.MP4"
videos = sorted(glob.glob(os.path.join(base_folder, video_name)))

# Should all frames be extracted? If False, specify target frame rate in fps (rounded to nearest integer)
extract_all_frames = True
target_frame_rate = 30
                
for video in videos:

    num_frames = 0

    print("[INFO] starting video file thread...")
    fvs = FileVideoStream(video).start()
    time.sleep(1.0)

    start_time = time.time()         
    video_name = os.path.basename(video).split('.')[0]

    output_folder = os.path.join(base_folder, "frames", video_name)
    os.makedirs(output_folder, exist_ok=True)

    if not extract_all_frames:
        vid = cv2.VideoCapture(video_file)
        fps = round(vid.get(cv2.CAP_PROP_FPS))
        vid.release()
        correction_factor = fps/target_frame_rate

    while fvs.more():
        frame = fvs.read()
        outfile = os.path.join(output_folder, 
                               f"{video_name}_{num_frames:06d}.jpg")
        if frame:
            if extract_all_frames:
                cv2.imwrite(outfile, frame)
            else:
                if num_frames % correction_factor == 0:
                    cv2.imwrite(outfile, frame)
        else:
            break

        if num_frames % 1000 == 0:
            print(f"{num_frames} processed, {np.any(frame)}")
        num_frames += 1
        
        # Uncomment below lines and specify value for max_frames if only
        # want to extract a subset of the frames from the video (maybe to save memory)
        # if num_frames > max_frames:
        #    break

    total_time = time.time() - start_time
    print("Total time: {:.2f}s".format(total_time))
    print(f"fps: {max_ims / total_time}")